Note the experiments in this are on the same prompts.
There is no baseline vs enhanced prompt

# Credential setup

Tried several things before finally getting credentials to work. Don't know if I need them all or if some are useless and redundant but here they are

In [1]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=ELLEcaMRoft2KovoeCaJajOOpoIL43&prompt=consent&token_usage=remote&access_type=offline&code_challenge=SWPHq_ECDUWwAsfTShnNkZjMWZr9BPvbJ5-ypETGNbA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0ATX87lNKtpT-72oluMjOQJXOyxwnxUCkRe-vmhOdcX41p7qP-RaVbTk898GtrSi18vUkpQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Q

In [2]:
! gcloud auth application-default set-quota-project v-480309


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "v-480309" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [3]:
!export GOOGLE_APPLICATION_CREDENTIALS="$HOME/.config/gcloud/application_default_credentials.json"

In [4]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth

    PROJECT_ID = "v-480309"
    LOCATION = "us-central1"

    # This will open a pop-up to log in and grant permissions.
    auth.authenticate_user(project_id=PROJECT_ID)

    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)

# Package setup

In [5]:
!pip install dspy-ai==3.0.3
!pip install crawl4ai==0.7.4
!pip install langchain_text_splitters==0.3.11
!pip install jupyter==1.0.0
!pip install nbconvert[webpdf]

!playwright install

  Using cached crawl4ai-0.7.4-py3-none-any.whl.metadata (39 kB)
  Using cached lxml-5.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.0/737.0 kB 39.0 MB/s eta 0:00:00
  Attempting uninstall: snowballstemmer
    Found existing instal

KeyboardInterrupt: 

In [5]:
import re
import dspy
from dataclasses import dataclass
from typing import List, Optional
import os
from dotenv import load_dotenv
import pandas as pd
import argparse
import json

In [7]:
import os
import json
from google.colab import userdata

## GET CREDENTIALS
## RUN ##
# !gcloud auth application-default login - run this to add vertex credentials to your env
## OR ##
"""
file_path = 'vertex_ai_service_account.json'

# Load the JSON file
with open(file_path, 'r') as file:
    vertex_credentials = json.load(file)

# Convert to JSON string
vertex_credentials_json = json.dumps(vertex_credentials)
os.environ["VERTEX_CREDENTIALS_JSON"] = vertex_credentials_json
"""
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Helper functions for single debate

In [8]:
"""
Crossover debate: Mix structured and baseline architectures.
Tests whether detailed prompting architecture provides an advantage.
"""

import dspy
from dataclasses import dataclass
from typing import List, Optional
import os
from dotenv import load_dotenv
import pandas as pd
import argparse
import json

load_dotenv()

# --- V5 PROMPTS ---

ARGUMENT_GENERATOR_PROMPT = """You are a divergent debate engine designed to build a constructive case.

**Phase 1: Brainstorming (Divergence)**
Generate **20 distinct argument fragments** for your side. Each fragment must have:
  * **Premise:** The starting condition.
  * **Mechanism:** The specific causal chain (How A leads to B).
  * **Impact:** The final outcome.
  * **Constraint:** The **MECHANISM must be unique** for every fragment. You may have similar impacts, but the logical path to get there must be different.

**Phase 2: Filtering (Contextual - 2nd Speakers Only)**
  * *If you are Prop 2 or Opp 2:* Read your teammate's previous speech.
  * **DELETE** any of your 20 fragments where the mechanism was already explained by your teammate. We need extensions, not repetition.

**Phase 3: Ranking (Strategic)**
  * Evaluate the remaining fragments. Rank them from 1 to N based on logical strength and strategic value.

**Phase 4: Clustering (Convergence)**
Group the top-ranked fragments into thematic arguments based on your role:
  * **If 1st Speaker (Prop 1 / Opp 1):** Take the top **6-10** fragments and cluster them into **2 distinct Contentions**.
  * **If 2nd Speaker (Prop 2 / Opp 2):** Take the top **3-5** fragments and cluster them into **1 single Extension Argument**.

**Output:** Return only the final clustered arguments with their taglines.
"""

SEMANTIC_PARSER_PROMPT = """You are a semantic classifier for debate motions.
Your goal is to map the motion to a high-level domain to determine which logic schemas to load.

Input Motion: "{motion}"

Task:
1. Identify the primary Domain from the following list: Economics, Politics & Governance, Law & Justice, Society & Culture, International Relations, Urban & Environment, Technology & AI, Education & Labor, Bioethics, Media (General).
2. Identify 3 key Stakeholders.

Output Format:
Domain: [Domain Name]
Stakeholders: [List of Stakeholders]
"""

SCHEMA_SLOT_FILLER_PROMPT = """You are a logic engine. Your goal is to apply an abstract argument template to a specific motion.

Motion: "{motion}"
Schema Name: "{schema_name}"
Logic Template: "{logic_template}"

Task:
Rewrite the Logic Template to apply specifically to this motion.
Replace abstract placeholders (like {{X}}, {{actor}}, {{policy}}) with specific terms from the motion.
Ensure the logic flows perfectly.

Output:
[A single paragraph argument based on the template]
"""

EXTRACTION_ENGINE_PROMPT = """You are an expert debate adjudicator and strategist. Your goal is to map the opponent's case into a structured hierarchy of threats.

**Step 1: Atomic Extraction**
Analyze the transcript(s). Extract distinct 'Strategic Ideas'. A Strategic Idea must be one of:
  * **Mechanism Flaw:** Why their action fails, won't happen, or won't lead to the result.
  * **Impact Claim:** Why their outcome matters (or explaining why it doesn't).
  * **Characterization:** How they define the world or the status quo.
    *Constraint:* Extract ONLY from the opponent's speech provided.

**Step 2: Strategic Ranking**
Rank these atomic ideas from 1 to N based on **'Damage Potential'**—which points are most likely to lose us the debate if left unanswered?

**Step 3: Thematic Clustering**
Group the top-ranked ideas into **{NUM_BUCKETS} thematic buckets**.
  * **If 2nd Speaker:** Create 2 Buckets.
  * **If 3rd Speaker:** Create 3 Buckets.

**Output Format:**
Return a structured object:
  * **Bucket 1 (Theme Name):**
      * **Idea 1:** [Description of Mechanism/Impact] (Rank: High)
      * **Idea 2:** [Description] (Rank: Medium)
  * **Bucket 2 (Theme Name):** ...
"""

REFUTATION_GENERATOR_PROMPT = """You are a ruthless debate strategist. Your goal is to destroy the opponent's case by attacking their specific ideas.

**Input:** A list of thematic buckets, where each bucket contains a list of specific 'Strategic Ideas'.
**Role:** You are the {SPEAKER_ROLE}.

**Instructions:**
Iterate through EVERY specific Idea in each Bucket. For **each Idea**, generate distinct responses based on your role intensity:
  * **If you are a 2nd Speaker:** Generate **2 distinct responses** for *each* Idea.
  * **If you are a 3rd Speaker:** Generate **3 distinct responses** for *each* Idea.

**Mandatory Tactics to Use:**
1.  **Worldview Challenge:** Rebut the underlying assumption (Characterization). 'They assume X, but the world is actually Y.'
2.  **The Turn (Flip):** Argue the mechanism causes the opposite effect or the impact actually helps YOUR side.
3.  **Destruction:** Direct denial of facts or mitigation of impact.

**Output:** A mapping:
  * Bucket -> Idea -> [Response 1, Response 2...]
"""

SPEECH_SYNTHESIZER_PROMPT = """You are the final speaker delivering a verbal debate speech. You have been given a set of 'Ingredients':
1.  **Refutations:** A map of responses to the enemy's points (if applicable).
2.  **Constructives:** New arguments/extensions to present (if applicable).

**Instructions:**
1.  Weave these ingredients into a cohesive, persuasive speech.
2.  **Structure:**
      * **Introduction:** Hook + Stance.
      * **Rebuttal Section:** Systematically go through the Refutations buckets. Use signposting (e.g., 'On their first clash about economics...').
      * **Constructive Section:** Present your New Arguments clearly (PEEL format).
      * **Conclusion:** Summarize why your side has won.
3.  Tone: Confident, engaging, and spoken-word style.
"""

# Load logic store
try:
    with open('logic_store.json', 'r') as f:
        LOGIC_STORE = json.load(f)
except FileNotFoundError:
    print("Warning: logic_store.json not found. Schema-guided architecture will fail.")
    LOGIC_STORE = []

BASELINE_PROMPTS = {
    "prop_1": "You are the First Proposition speaker in a 3v3 debate. Your speech should be approximately 1200 words. Do not use greetings or pleasantries. Please define the motion and present the main constructive arguments for the Proposition side.",
    "opp_1": "You are the First Opposition speaker in a 3v3 debate. Your speech should be approximately 1200 words. Do not use greetings or pleasantries. Please rebut the First Proposition speaker and present the main constructive arguments for the Opposition side.",
    "prop_2": "You are the Second Proposition speaker in a 3v3 debate. Your speech should be approximately 1200 words. Do not use greetings or pleasantries. Please rebut the Opposition, defend your partner's arguments, and introduce a new constructive argument.",
    "opp_2": "You are the Second Opposition speaker in a 3v3 debate. Your speech should be approximately 1200 words. Do not use greetings or pleasantries. Please rebut the Proposition, defend your partner's arguments, and introduce a new constructive argument.",
    "prop_3": "You are the Third Proposition speaker in a 3v3 debate. Your speech should be approximately 1200 words. Do not use greetings or pleasantries. Please summarize the debate and explain why the Proposition team has won. Do not introduce new constructive arguments.",
    "opp_3": "You are the Third Opposition speaker in a 3v3 debate. Your speech should be approximately 1200 words. Do not use greetings or pleasantries. Please summarize the debate and explain why the Opposition team has won. Do not introduce new constructive arguments."
}

@dataclass
class Turn:
    speaker_position: str
    team: str
    speaker_number: int
    speech: str
    architecture: str

@dataclass
class DebateResult:
    motion: str
    prop_model: str
    opp_model: str
    judge_model: str
    prop_architecture: str
    opp_architecture: str
    turns: List[Turn]
    prop_scores: List[int]
    opp_scores: List[int]
    winner: str
    reason_for_decision: str

class DebateSpeaker:
    def __init__(self, lm, speaker_position: str, architecture: str):
        self.lm = lm
        self.speaker_position = speaker_position
        self.architecture = architecture
        self.team = "Proposition" if "prop" in speaker_position else "Opposition"
        self.speaker_number = int(speaker_position.split('_')[1])

        # Map position to role name
        roles = {
            "prop_1": "Prime Minister",
            "opp_1": "Leader of Opposition",
            "prop_2": "Deputy Prime Minister",
            "opp_2": "Deputy Leader of Opposition",
            "prop_3": "Government Whip",
            "opp_3": "Opposition Whip"
        }
        self.role_name = roles.get(speaker_position, "Debater")

    def _call_llm(self, prompt: str) -> str:
        with dspy.context(lm=self.lm):
            response = self.lm(prompt=prompt)
            if isinstance(response, list):
                return response[0] if response else ""
            return response

    def generate_speech(self, motion: str, teammate_speech: Optional[str], opponent_speeches: List[str]) -> str:
        if self.architecture == "baseline":
            return self._generate_baseline(motion, teammate_speech, opponent_speeches)
        elif self.architecture == "schema_guided":
            return self._generate_schema_guided(motion, teammate_speech, opponent_speeches)
        else:
            return self._generate_enhanced(motion, teammate_speech, opponent_speeches)

    def _generate_baseline(self, motion: str, teammate_speech: Optional[str], opponent_speeches: List[str]) -> str:
        prompt = BASELINE_PROMPTS[self.speaker_position]
        context = f"Motion: {motion}\n\n"
        if teammate_speech:
            context += f"Your Partner's Speech:\n{teammate_speech}\n\n"
        if opponent_speeches:
            context += "Opponent Speeches:\n"
            for i, speech in enumerate(opponent_speeches):
                context += f"Opponent {i+1}:\n{speech}\n\n"

        full_prompt = f"{context}{prompt}\n\nYour Speech:"
        return self._call_llm(full_prompt)

    def _generate_schema_guided(self, motion: str, teammate_speech: Optional[str], opponent_speeches: List[str]) -> str:
        # --- STEP 1: EXTRACTION & RANKING (Same as Enhanced) ---
        clustered_threats = ""
        if self.speaker_number in [2, 3]:
            num_buckets = 2 if self.speaker_number == 2 else 3
            opp_transcript = "\n\n".join(opponent_speeches)

            extraction_prompt = EXTRACTION_ENGINE_PROMPT.format(NUM_BUCKETS=num_buckets)
            extraction_input = f"TRANSCRIPT TO ANALYZE:\n{opp_transcript}\n\nINSTRUCTIONS:\n{extraction_prompt}"

            # print(f"    [Extraction Layer Active]")
            clustered_threats = self._call_llm(extraction_input)

        # --- STEP 2: REFUTATION (Same as Enhanced) ---
        refutations_map = ""
        if clustered_threats:
            refutation_prompt = REFUTATION_GENERATOR_PROMPT.format(SPEAKER_ROLE=self.role_name)
            refutation_input = f"THREATS TO DESTROY:\n{clustered_threats}\n\nINSTRUCTIONS:\n{refutation_prompt}"

            # print(f"    [Refutation Layer Active]")
            refutations_map = self._call_llm(refutation_input)

        # --- STEP 3: SCHEMA-GUIDED GENERATION ---
        new_case = ""
        if self.speaker_number in [1, 2]:
            # 1. Semantic Parsing
            parser_prompt = SEMANTIC_PARSER_PROMPT.format(motion=motion)
            # print(f"    [Semantic Parsing Active]")
            parsing_result = self._call_llm(parser_prompt)

            # Extract Domain
            domain = "Economics" # Default
            for line in parsing_result.split('\n'):
                if "Domain:" in line:
                    domain = line.split("Domain:")[1].strip()
                    break

            # print(f"    [Logic Retrieval Active] Domain: {domain}")

            # 2. Logic Retrieval
            schemas = []
            # Fuzzy match domain
            for entry in LOGIC_STORE:
                if entry["domain"].lower() in domain.lower() or domain.lower() in entry["domain"].lower():
                    schemas = entry["mechanisms"]
                    break

            if not schemas:
                # Fallback
                print(f"    [Warning] Domain not found in Logic Store. Using default.")
                schemas = LOGIC_STORE[0]["mechanisms"]

            # 3. Slot Filling
            # print(f"    [Schema-Guided Generation Active] Applying {len(schemas)} schemas...")
            generated_arguments = []
            for schema in schemas:
                filler_prompt = SCHEMA_SLOT_FILLER_PROMPT.format(
                    motion=motion,
                    schema_name=schema["name"],
                    logic_template=schema["logic_template"]
                )
                argument = self._call_llm(filler_prompt)
                generated_arguments.append(f"### Argument: {schema['name']}\n{argument}")

            new_case = "\n\n".join(generated_arguments)

        # --- STEP 4: SYNTHESIS ---
        synthesis_input = f"Motion: {motion}\nRole: {self.role_name}\n"
        if refutations_map:
            synthesis_input += f"\nREFUTATION INGREDIENTS:\n{refutations_map}\n"
        if new_case:
            synthesis_input += f"\nCONSTRUCTIVE INGREDIENTS:\n{new_case}\n"

        synthesis_input += f"\nINSTRUCTIONS:\n{SPEECH_SYNTHESIZER_PROMPT}"

        print(f"    [Speech Synthesizer Active]")
        return self._call_llm(synthesis_input)

    def _generate_enhanced(self, motion: str, teammate_speech: Optional[str], opponent_speeches: List[str]) -> str:
        # --- STEP 1: EXTRACTION & RANKING ---
        clustered_threats = ""
        if self.speaker_number in [2, 3]:
            num_buckets = 2 if self.speaker_number == 2 else 3
            opp_transcript = "\n\n".join(opponent_speeches)

            extraction_prompt = EXTRACTION_ENGINE_PROMPT.format(NUM_BUCKETS=num_buckets)
            extraction_input = f"TRANSCRIPT TO ANALYZE:\n{opp_transcript}\n\nINSTRUCTIONS:\n{extraction_prompt}"

            # print(f"    [Extraction Layer Active]")
            clustered_threats = self._call_llm(extraction_input)

        # --- STEP 2: REFUTATION ---
        refutations_map = ""
        if clustered_threats:
            refutation_prompt = REFUTATION_GENERATOR_PROMPT.format(SPEAKER_ROLE=self.role_name)
            refutation_input = f"THREATS TO DESTROY:\n{clustered_threats}\n\nINSTRUCTIONS:\n{refutation_prompt}"

            # print(f"    [Refutation Layer Active]")
            refutations_map = self._call_llm(refutation_input)

        # --- STEP 3: GENERATION (Case Building) ---
        new_case = ""
        if self.speaker_number in [1, 2]:
            # Prop 1, Opp 1, Prop 2, Opp 2

            generation_input = f"Motion: {motion}\nSide: {self.team}\nRole: {self.role_name}\n"
            if teammate_speech:
                generation_input += f"Teammate's Previous Speech:\n{teammate_speech}\n"

            generation_input += f"\nINSTRUCTIONS:\n{ARGUMENT_GENERATOR_PROMPT}"

            print(f"    [Generation Layer Active]")
            new_case = self._call_llm(generation_input)

        # --- STEP 4: SYNTHESIS ---
        synthesis_input = f"Motion: {motion}\nRole: {self.role_name}\n"
        if refutations_map:
            synthesis_input += f"\nREFUTATION INGREDIENTS:\n{refutations_map}\n"
        if new_case:
            synthesis_input += f"\nCONSTRUCTIVE INGREDIENTS:\n{new_case}\n"

        synthesis_input += f"\nINSTRUCTIONS:\n{SPEECH_SYNTHESIZER_PROMPT}"

        print(f"    [Speech Synthesizer Active]")
        return self._call_llm(synthesis_input)

def judge_debate(motion: str, turns: List[Turn], judge_lm) -> tuple:
    """Judge the debate with strict OIV criteria and score all 6 speakers."""

    transcript = f"Motion: {motion}\n\n"
    for turn in turns:
        transcript += f"\n--- {turn.team} Speaker {turn.speaker_number} ---\n{turn.speech}\n"

    judge_prompt = f"""You are an Expert Debate Adjudicator

You are tasked with judging a 3v3 debate between Proposition and Opposition teams. You must adopt the persona of the Ordinary Intelligent Voter (OIV) as defined by the WUDC Debating Manual.

Your Persona & Disposition
* Knowledge Base: You are a smart, generalist reader. You know basic geopolitical and social facts (e.g., you know Syria is in the Middle East), but you do not possess specialist or technical knowledge (e.g., you do not know specific legal precedents or complex economic formulas).
* Open-Mindedness: You have no preformed views on the topic. You are cynical about mere assertions and require reasoned analysis to be persuaded.
* Neutrality: You must not judge based on your personal preferences or political alignment. You judge solely on the persuasiveness of the arguments presented within the text.

Your Evaluation Process
You must evaluate the debate based on four weighted axes.

1. Argumentation & Analysis (45%)
* Logic over Assertion: Do not credit bare assertions. Credit arguments that provide mechanistic links—reasons why X leads to Y.
* Impact: Credit teams that explain why an outcome matters (morally, practically, or emotionally).
* Consistency: Check for contradictions. If a later speaker contradicts an earlier partner, ignore the later claim and stick to the team's original stance.
* Plausibility: Reject claims that are factually absurd or logical leaps that an ordinary person would find impossible to believe.

2. Engagement & Rebuttal (35%)
* Direct Responsiveness: You must track which arguments were answered. If a team ignores a core argument from their opponents, you must treat that argument as conceded and true.
* The Silence Rule: If Proposition proves X is true and Opposition never mentions X, then X is a fact in this debate. You must weigh this fact in your final decision.
* Comparative Weighing: Give higher credit to teams that explicitly compare their impacts against their opponents' (e.g., Our impact affects fewer people but is a breach of a more important human right).

3. Role Fulfillment (10%)
* 1st Proposition: Must define the motion. If the definition is a squirrel (unfairly restrictive or logically impossible), penalize them heavily.
* 1st Opposition: Must oppose the motion. They may defend the status quo or propose a counter-model.

4. Clarity of Expression (10%)
* Precision: Evaluate whether the meaning of the text is unambiguous.
* Comprehensibility: The text must be clear enough for an average reader to follow the logic.
* Note on Jargon: While you should not penalize the use of technical terms heavily, if a term renders an argument confusing to a layperson, treat the argument as less persuasive.

Scoring Guide (Strict Adherence Required)

You must assign a score between 50 and 100 to each speech. Be extremely strict. Do not succumb to grade inflation.

* 90-100 (Legendary/Rare): Do not award this lightly. This score is reserved for the best speeches in human history. The argument is flawless, the rebuttal is devastating, and it is nearly impossible to imagine a better speech.
* 85-89 (Exceptional): A very high bar. The speech addresses the core issues with sophisticated analysis and has almost no flaws.
* 80-84 (Very Good): A high-quality speech. It is relevant, deeply analytical, and well-structured, but may have minor vulnerabilities.
* 75-79 (Good/Competent): Relevant and logical, but may rely on some simplifications or miss minor nuances.
* 70-74 (Beginner/Average): The speech is relevant but has significant logical gaps or relies on assertions rather than proof.
* 60-69 (Below Average/Poor): Hard to follow, barely relevant, or logically confused.
* 50-59 (Non-Functional): Irrelevant content or gibberish.

Step-by-Step Decision Protocol

1. Identify Claims: List the core arguments made by Proposition and Opposition.
2. Filter Invalid Arguments:
* Delete arguments based on knifing (contradicting partners).
* Delete assertions that lack logical backing.
3. Determine Truth:
* Did the Opponent rebut Claim A?
* If YES: Evaluate who had better analysis.
* If NO: Claim A is true.
4. Weigh Impacts: Compare the surviving impacts. Use the metrics provided by the teams. If no metrics were provided, use your common sense as an Ordinary Intelligent Voter.
5. Score Each Speaker: Assign a score between 50-100 to each speaker based on the criteria above.
6. Calculate Total Scores: Sum all speaker scores for each team.
7. Declare Winner: THE TEAM WITH THE HIGHER TOTAL SCORE WINS. This is a mathematical determination based on the scores you assigned. If Proposition's total score is higher, Proposition wins. If Opposition's total score is higher, Opposition wins. You must follow this rule strictly.

--- DEBATE TRANSCRIPT ---
{transcript}
--- END TRANSCRIPT ---

Provide your evaluation in this EXACT format:

PROPOSITION SPEAKER 1 SCORE: [score]
OPPOSITION SPEAKER 1 SCORE: [score]
PROPOSITION SPEAKER 2 SCORE: [score]
OPPOSITION SPEAKER 2 SCORE: [score]
PROPOSITION SPEAKER 3 SCORE: [score]
OPPOSITION SPEAKER 3 SCORE: [score]
WINNER: [Proposition/Opposition]
REASON: [Your detailed RFD explaining why this team won]
"""

    with dspy.context(lm=judge_lm):
        response = judge_lm(prompt=judge_prompt)

    if isinstance(response, list):
        response = response[0] if response else ""

    lines = response.strip().split('\n')

    # Initialize dictionaries to map specific speakers to scores (robust against skipped speakers)
    p_scores_map = {1: None, 2: None, 3: None}
    o_scores_map = {1: None, 2: None, 3: None}

    reason_lines = []
    capturing_reason = False

    for line in lines:
        lower_line = line.lower()
        # Parse Proposition Scores with speaker number extraction
        if "proposition speaker" in lower_line and "score:" in lower_line:
            try:
                parts = lower_line.split("proposition speaker")[1].split("score:")
                speaker_num = int(parts[0].strip())  # Extract '1' from 'proposition speaker 1 score:'
                score = int(parts[1].strip().split()[0])
                if speaker_num in p_scores_map:
                    p_scores_map[speaker_num] = score
            except:
                pass
        # Parse Opposition Scores with speaker number extraction
        elif "opposition speaker" in lower_line and "score:" in lower_line:
            try:
                parts = lower_line.split("opposition speaker")[1].split("score:")
                speaker_num = int(parts[0].strip())
                score = int(parts[1].strip().split()[0])
                if speaker_num in o_scores_map:
                    o_scores_map[speaker_num] = score
            except:
                pass
        elif line.startswith("REASON:"):
            reason_lines.append(line.split("REASON:")[1].strip())
            capturing_reason = True
        elif capturing_reason and line.strip() and not line.upper().startswith("WINNER"):
            reason_lines.append(line.strip())

    # Flatten maps to lists (preserving order, filtering None)
    prop_scores = [p_scores_map[i] for i in range(1, 4) if p_scores_map[i] is not None]
    opp_scores = [o_scores_map[i] for i in range(1, 4) if o_scores_map[i] is not None]

    # Calculate Winner Mathematically (ignoring LLM's text claim for data integrity)
    p_total = sum(prop_scores) if prop_scores else 0
    o_total = sum(opp_scores) if opp_scores else 0

    if p_total > o_total:
        winner = "Proposition"
    elif o_total > p_total:
        winner = "Opposition"
    else:
        winner = "Tie"

    reason = " ".join(reason_lines) if reason_lines else ""

    return prop_scores, opp_scores, winner, reason


def run_crossover_debate(motion: str,
                        prop_architecture: str,
                        opp_architecture: str,
                        prop_model_name: str,
                        opp_model_name: str,
                        judge_model_name: str,
                        prop_lm,
                        opp_lm,
                        judge_lm,
                        num_turns: int = 3) -> DebateResult:

    print(f"\n=== Running Crossover Debate ({num_turns}v{num_turns}) ===")
    print(f"Motion: {motion}")
    print(f"Proposition: {prop_model_name} ({prop_architecture} architecture)")
    print(f"Opposition: {opp_model_name} ({opp_architecture} architecture)")
    print(f"Judge: {judge_model_name}\n")

    turns = []
    prop_speeches = []
    opp_speeches = []

    # Full speaking order for 3v3
    full_speaking_order = [
        ("prop_1", "Proposition", 1),
        ("opp_1", "Opposition", 1),
        ("prop_2", "Proposition", 2),
        ("opp_2", "Opposition", 2),
        ("prop_3", "Proposition", 3),
        ("opp_3", "Opposition", 3)
    ]

    # Truncate based on num_turns
    speaking_order = full_speaking_order[:num_turns * 2]

    for position, team, speaker_num in speaking_order:
        arch = prop_architecture if team == "Proposition" else opp_architecture
        lm = prop_lm if team == "Proposition" else opp_lm

        speaker = DebateSpeaker(lm, position, arch)

        teammate_speech = None
        opponent_speeches = []

        if team == "Proposition":
            if prop_speeches:
                teammate_speech = prop_speeches[-1]
            opponent_speeches = opp_speeches
        else:
            if opp_speeches:
                teammate_speech = opp_speeches[-1]
            opponent_speeches = prop_speeches

        print(f"  {team} Speaker {speaker_num} generating speech...")
        speech = speaker.generate_speech(motion, teammate_speech, opponent_speeches)

        if team == "Proposition":
            prop_speeches.append(speech)
        else:
            opp_speeches.append(speech)

        turns.append(Turn(position, team, speaker_num, speech, arch))
        print(f"  {team} Speaker {speaker_num} spoke.")

    print("\n  Judging debate...")
    prop_scores, opp_scores, winner, rfd = judge_debate(motion, turns, judge_lm)
    print(f"  Winner: {winner}\n")

    return DebateResult(
        motion=motion,
        prop_model=prop_model_name,
        opp_model=opp_model_name,
        judge_model=judge_model_name,
        prop_architecture=prop_architecture,
        opp_architecture=opp_architecture,
        turns=turns,
        prop_scores=prop_scores,
        opp_scores=opp_scores,
        winner=winner,
        reason_for_decision=rfd
    )


# Initialize models

In [9]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
  raise ValueError("OPENAI_API_KEY not found in environment")

"""
vertex_credentials_json = os.getenv("VERTEX_CREDENTIALS_JSON")
if not vertex_credentials_json:
  raise ValueError("You must set VERTEX_CREDENTIALS_JSON in your environment")

gemini_flash = dspy.LM(
    "vertex_ai/" + "gemini-2.5-flash",
    vertex_credentials=vertex_credentials_json
)
gemini_pro = dspy.LM(
    "vertex_ai/" + "gemini-2.5-pro",
    vertex_credentials=vertex_credentials_json
)
qwen = dspy.LM(
    "vertex_ai/" + "qwen_qwen3-0_6b-1765155687846",
    vertex_credentials=vertex_credentials_json
)
"""
gemini_flash = dspy.LM(
    "vertex_ai/" + "gemini-2.5-flash",
)
gemini_pro = dspy.LM(
    "vertex_ai/" + "gemini-2.5-pro",
)
qwen = dspy.LM(
    "vertex_ai/" + "qwen_qwen3-0_6b-1765155687846",
)
llama_maverick = dspy.LM(
    "vertex_ai/" + "llama-maverick-7b-chat",
)
o3 = dspy.LM(f'openai/o3', api_key=api_key)

In [10]:
# Return output so it can be added to dataframe. Do not put output in CSV
# 'enhanced', 'baseline', 'schema_guided'
def run_debate(prop_model, opp_model, judge_model, prop_arch, opp_arch, motion, turns=3):
    map = {
        "flash": gemini_flash,
        "pro": gemini_pro,
        "qwen": qwen,
        "o3": o3
    }

    prop_lm = map[prop_model]
    opp_lm = map[opp_model]
    judge_lm = map[judge_model]

    print("\n" + "="*80)
    print(f"MATCHUP: {prop_arch} Prop ({prop_model}) vs {opp_arch} Opp ({opp_model})")
    print("="*80)

    result = run_crossover_debate(
        motion=motion,
        prop_architecture=prop_arch,
        opp_architecture=opp_arch,
        prop_model_name=prop_model,
        opp_model_name=opp_model,
        judge_model_name=judge_model,
        prop_lm=prop_lm,
        opp_lm=opp_lm,
        judge_lm=judge_lm,
        num_turns=turns
    )

    return result

In [20]:
# Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error.
# Run $ gcloud auth application-default set-quota-project to add a quota project.

# Experiments

In [17]:
motions= ["This House Believes that developing countries should prioritise service led economic growth over manufacturing led economic growth.",
          "This House regrets the norm of association between sex and romantic love.",
          "This House believes that China should pursue aggressive economic stimulus policies (e.g., injecting central bank funds directly into the economy at large scale, providing stimulus checks)."
          "This House Opposes the narrative that power tends to corrupt.",
          "This House opposes the expectation that romantic partners should be significant support systems of each other's mental health.",
          "This House would heavily ease labour regulations in times of economic crisis (e.g., heavily reducing/removing the minimum wage, relaxing safety laws, etc.).",
          "This House regrets the glorification of the career longevity of dominant sportspersons (e.g., Lebron James, Tom Brady, Cristiano Ronaldo).",
          "This House believes that works of modern fictional media should not portray members of an oppressed minority group (such as LGBTQ+ or minority ethnicity groups) as villains.",
          "This House opposes the norm to prefer the natural to the artificial.",
          "This House believes that education systems should over-inflate children's academic self-perception (e.g., providing overwhelmingly positive feedback, avoiding fail grades, etc.)"
          ]
models = ["flash", "pro"]
len(motions)


9

In [36]:
# Baseline, schema-guided, enhanced
# Model competes against the "improved" version of itself
pairs = [("baseline", "schema-guided"),
         ("baseline", "enhanced"),
         ("schema-guided", "enhanced")
]
"""
class DebateResult:
    motion: str
    prop_model: str
    opp_model: str
    judge_model: str
    prop_architecture: str
    opp_architecture: str
    turns: List[Turn]
    prop_scores: List[int]
    opp_scores: List[int]
    winner: str
    reason_for_decision: str
"""
results = []
for motion in motions:
  for model in models:
    for pair in pairs:
      result_dict = {}
      result_dict["Motion"] = motion
      result_dict["Model"] = model
      result_dict["First"] = pair[0]
      result_dict["Second"] = pair[1]

      result1 = run_debate(model, model, "o3", pair[0], pair[1], motion)
      result2 = run_debate(model, model, "o3", pair[1], pair[0], motion)

      result_dict["Forward winner"] = result1.winner
      result_dict["Reverse winner"] = result2.winner

      # Pair in order: Model 1 is prop, model 2 is opp
      result_dict["First prop scores"] = result1.prop_scores
      result_dict["Second opp scores"] = result1.opp_scores
      # Reverse order: Model 2 is now prop, model 1 is now opp
      result_dict["Second prop scores"] = result2.prop_scores
      result_dict["First opp scores"] = result2.opp_scores

      result_dict["Forward turn"] = result1.turns
      result_dict["Reverse turn"] = result2.turns
      result_dict["Forward reason"] = result1.reason_for_decision
      result_dict["Reverse reason"] = result2.reason_for_decision

      results.append(result_dict)


MATCHUP: baseline Prop (flash) vs schema-guided Opp (flash)

=== Running Crossover Debate (3v3) ===
Motion: This House Believes that developing countries should prioritise service led economic growth over manufacturing led economic growth.
Proposition: flash (baseline architecture)
Opposition: flash (schema-guided architecture)
Judge: o3

  Proposition Speaker 1 generating speech...
  Proposition Speaker 1 spoke.
  Opposition Speaker 1 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Opposition Speaker 1 spoke.
  Proposition Speaker 2 generating speech...
  Proposition Speaker 2 spoke.
  Opposition Speaker 2 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Opposition Speaker 2 spoke.
  Proposition Speaker 3 generating speech...
  Proposition Speaker 3 spoke.
  Opposition Speaker 3 generating speech...
    [Speech Synthesizer Active]
  Opposition Speaker 3 spoke.

  Judging debate...
  Winner: Proposition


MATCHUP

2025/12/08 09:49:12 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=None. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently None)  if the reason for truncation is repetition.


    [Speech Synthesizer Active]
  Opposition Speaker 2 spoke.
  Proposition Speaker 3 generating speech...
  Proposition Speaker 3 spoke.
  Opposition Speaker 3 generating speech...
    [Speech Synthesizer Active]
  Opposition Speaker 3 spoke.

  Judging debate...
  Winner: Proposition


MATCHUP: schema-guided Prop (pro) vs baseline Opp (pro)

=== Running Crossover Debate (3v3) ===
Motion: This House would heavily ease labour regulations in times of economic crisis (e.g., heavily reducing/removing the minimum wage, relaxing safety laws, etc.).
Proposition: pro (schema-guided architecture)
Opposition: pro (baseline architecture)
Judge: o3

  Proposition Speaker 1 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Proposition Speaker 1 spoke.
  Opposition Speaker 1 generating speech...
  Opposition Speaker 1 spoke.
  Proposition Speaker 2 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Proposition Speaker 2 spoke.
  O

2025/12/08 10:03:41 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=None. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently None)  if the reason for truncation is repetition.


  Winner: Opposition


MATCHUP: baseline Prop (pro) vs enhanced Opp (pro)

=== Running Crossover Debate (3v3) ===
Motion: This House would heavily ease labour regulations in times of economic crisis (e.g., heavily reducing/removing the minimum wage, relaxing safety laws, etc.).
Proposition: pro (baseline architecture)
Opposition: pro (enhanced architecture)
Judge: o3

  Proposition Speaker 1 generating speech...
  Proposition Speaker 1 spoke.
  Opposition Speaker 1 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Opposition Speaker 1 spoke.
  Proposition Speaker 2 generating speech...
  Proposition Speaker 2 spoke.
  Opposition Speaker 2 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Opposition Speaker 2 spoke.
  Proposition Speaker 3 generating speech...
  Proposition Speaker 3 spoke.
  Opposition Speaker 3 generating speech...
    [Speech Synthesizer Active]
  Opposition Speaker 3 spoke.

  Judging debate...
  

2025/12/08 10:06:52 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=None. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently None)  if the reason for truncation is repetition.


    [Generation Layer Active]
    [Speech Synthesizer Active]
  Opposition Speaker 2 spoke.
  Proposition Speaker 3 generating speech...
    [Speech Synthesizer Active]
  Proposition Speaker 3 spoke.
  Opposition Speaker 3 generating speech...
    [Speech Synthesizer Active]
  Opposition Speaker 3 spoke.

  Judging debate...


2025/12/08 10:12:19 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=None. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently None)  if the reason for truncation is repetition.


  Winner: Opposition


MATCHUP: enhanced Prop (pro) vs schema-guided Opp (pro)

=== Running Crossover Debate (3v3) ===
Motion: This House would heavily ease labour regulations in times of economic crisis (e.g., heavily reducing/removing the minimum wage, relaxing safety laws, etc.).
Proposition: pro (enhanced architecture)
Opposition: pro (schema-guided architecture)
Judge: o3

  Proposition Speaker 1 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Proposition Speaker 1 spoke.
  Opposition Speaker 1 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Opposition Speaker 1 spoke.
  Proposition Speaker 2 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Proposition Speaker 2 spoke.
  Opposition Speaker 2 generating speech...
    [Generation Layer Active]
    [Speech Synthesizer Active]
  Opposition Speaker 2 spoke.
  Proposition Speaker 3 generating speech...
    [Speech Synthesizer Ac

In [37]:
df = pd.DataFrame(results)

In [38]:
df.to_csv('output.csv', index=False)

In [39]:
from google.colab import files
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sanity check experiment

In [12]:
# Sanity check experiment between baseline regular (2.5) and baseline mini (2.5-flash)
"""
class DebateResult:
    motion: str
    prop_model: str
    opp_model: str
    judge_model: str
    prop_architecture: str
    opp_architecture: str
    turns: List[Turn]
    prop_scores: List[int]
    opp_scores: List[int]
    winner: str
    reason_for_decision: str
"""
check_results = []
for i in range(len(motions)):
  motion = motions[i]
  result_dict = {}
  result_dict["Motion"] = motion
  if i % 2 == 0: # Half of the time, let flash be prop
    result_dict["Flash role"] = "Proposition"
    result_dict["Prop role"] = "Opposition"
    result = run_debate("flash", "pro", "o3", "baseline", "baseline", motion)
    if result.winner == "Proposition":
      result_dict["Winner"] = "Flash"
    elif result.winner == "Opposition":
      result_dict["Winner"] = "Pro"
    result_dict["Flash scores"] = result.prop_scores
    result_dict["Pro scores"] = result.opp_scores
    result_dict["Turns"] = result.turns
    result_dict["Reason"] = result.reason_for_decision

  else: # Rest of the time, let pro be prop
    result_dict["Flash role"] = "Opposition"
    result_dict["Prop role"] = "Proposition"
    result = run_debate("pro", "flash", "o3", "baseline", "baseline", motion)
    if result.winner == "Proposition":
      result_dict["Winner"] = "Pro"
    elif result.winner == "Opposition":
      result_dict["Winner"] = "Flash"
    result_dict["Flash scores"] = result.opp_scores
    result_dict["Pro scores"] = result.prop_scores
    result_dict["Turns"] = result.turns
    result_dict["Reason"] = result.reason_for_decision

  check_results.append(result_dict)


MATCHUP: baseline Prop (flash) vs baseline Opp (pro)

=== Running Crossover Debate (3v3) ===
Motion: This House Believes that developing countries should prioritise service led economic growth over manufacturing led economic growth.
Proposition: flash (baseline architecture)
Opposition: pro (baseline architecture)
Judge: o3

  Proposition Speaker 1 generating speech...
  Proposition Speaker 1 spoke.
  Opposition Speaker 1 generating speech...
  Opposition Speaker 1 spoke.
  Proposition Speaker 2 generating speech...
  Proposition Speaker 2 spoke.
  Opposition Speaker 2 generating speech...
  Opposition Speaker 2 spoke.
  Proposition Speaker 3 generating speech...
  Proposition Speaker 3 spoke.
  Opposition Speaker 3 generating speech...
  Opposition Speaker 3 spoke.

  Judging debate...
  Winner: Opposition


MATCHUP: baseline Prop (pro) vs baseline Opp (flash)

=== Running Crossover Debate (3v3) ===
Motion: This House regrets the norm of association between sex and romantic love.
Pro

In [13]:
df_check = pd.DataFrame(check_results)

In [14]:
df_check.to_csv('sanitycheck.csv', index=False)

In [15]:
from google.colab import files
files.download('sanitycheck.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
df_check

,Motion,Flash role,Prop role,Winner,Flash scores,Pro scores,Turns,Reason
0,This House Believes that developing countries ...,Proposition,Opposition,Pro,"[78, 76, 74]","[80, 82, 79]","[Turn(speaker_position='prop_1', team='Proposi...",1. Employment & Inclusivity (largest clash) •...
1,This House regrets the norm of association bet...,Opposition,Proposition,Pro,"[79, 77, 76]","[82, 83, 80]","[Turn(speaker_position='prop_1', team='Proposi...",1. Core Clash – Is harm baked into the norm o...
2,This House believes that China should pursue a...,Proposition,Opposition,Pro,"[77, 75, 72]","[82, 80, 78]","[Turn(speaker_position='prop_1', team='Proposi...",1. Argumentation & Analysis (45 %) • Economic...
3,This House opposes the expectation that romant...,Opposition,Proposition,Pro,"[80, 77, 76]","[80, 79, 78]","[Turn(speaker_position='prop_1', team='Proposi...",1. Definition & Burden of Proof – Prop 1 set ...
4,This House would heavily ease labour regulatio...,Proposition,Opposition,Pro,"[76, 78, 77]","[80, 82, 81]","[Turn(speaker_position='prop_1', team='Proposi...",1. Central Clash – Aggregate Demand vs. Labou...
5,This House regrets the glorification of the ca...,Opposition,Proposition,Pro,"[77, 75, 74]","[78, 76, 77]","[Turn(speaker_position='prop_1', team='Proposi...","1. Definition & Burden – Prop 1 gave a clear,..."
6,This House believes that works of modern ficti...,Proposition,Opposition,Pro,"[77, 75, 72]","[80, 78, 77]","[Turn(speaker_position='prop_1', team='Proposi...",1. Argumentation & Analysis (45%) • Proposit...
7,This House opposes the norm to prefer the natu...,Opposition,Proposition,Pro,"[79, 78, 77]","[82, 80, 78]","[Turn(speaker_position='prop_1', team='Proposi...",1. Argumentation & Analysis • Proposition set...
8,This House believes that education systems sho...,Proposition,Opposition,Pro,"[78, 75, 74]","[81, 80, 78]","[Turn(speaker_position='prop_1', team='Proposi...",1. Argumentation & Analysis • Learning Outcom...


## Planning

In [ ]:
"""
First test three times, one each on a different Vertex model to make sure the command line interface works

Gemini 2.5 Pro vs Gemini 2.5 Flash
Claude Sonnet 4.5 vs Claude Haiku 4.5 (mini)
Qwen3

gemini-2.5-flash, gemini-2.5-pro, qwen_qwen3-0_6b-1765155687846,

"""

"""General plan:


For each of the 10 motions:

  For each of the 3 models (Gemini and???): --- Nevevr mind, just different Geminis. Generation is slow, so don't have time to test further

    For regular size, mini:

      Baseline vs prompted of same model (and reverse)
      Baseline vs preprocessed (and reverse)
      Baseline vs schematic (and reverse)

      Prompted vs preprocessed (and reverse)
      prompted vs schematic (and reverse)

      Preprocessed vs schematic (and reverse)
20 x 3 x 2 x 6 x 2 = 240 debates
"""


""" Another plan, sanity check:
For 5 selectd motions:
  For each of th 3 models:
    Baseline vs Baseline mini (and reverse)
5x3x2 = 30 debates
"""
